In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#import scipy.stats as stats
#import statsmodels.api as sm
import re
import plotly.express as px
import requests
#from bs4 import BeautifulSoup as BS
from io import StringIO
#from shapely.geometry import Point
#from scipy.spatial import distance
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import math
#import geopy.distance
#from tqdm.notebook import tqdm
%matplotlib inline
pd.set_option('display.max_columns', None)  

In [2]:
covid_data = pd.read_csv('Merged_google_Covid_cases.csv')

In [3]:
covid_data

,Unnamed: 0,mapped_location,lat,lng,address,results,business_name1,business_vicinity1,business_type1,business_name2,business_vicinity2,business_type2,business_name3,business_vicinity3,business_type3,business_name4,business_vicinity4,business_type4,business_name5,business_vicinity5,business_type5,address_0,address_1,address_2,address_3,address_4,address_5,sum,final_name,final_vicinity,final_type,ratio,address_city,Request #
0,0,"(36.0685457, -86.6838975)",36.068546,-86.683898,"2045 Antioch Pike, Antioch","[{'business_status': 'OPERATIONAL', 'geometry'...",Los Paisanos,"2045 Antioch Pike, Antioch","['night_club', 'bar', 'point_of_interest', 'es...",Sun Hair Salon,"2049 Antioch Pike, Antioch","['beauty_salon', 'hair_care', 'point_of_intere...",Solo Style LLC,"2037 Antioch Pike, Antioch","['clothing_store', 'point_of_interest', 'store...",Hai Woon Dai,"2051 Antioch Pike, Antioch","['restaurant', 'food', 'point_of_interest', 'e...","Dr. Babajide A. Bamigboye, MD","2031 Antioch Pike, Antioch","['doctor', 'health', 'point_of_interest', 'est...",2045,True,False,False,False,False,1,Los Paisanos,"2045 Antioch Pike, Antioch","['night_club', 'bar', 'point_of_interest', 'es...",100,"2045 Antioch Pike, Antioch",8
1,1,"(36.1941628, -86.8313936)",36.194163,-86.831394,"3200 Clarksville Pike, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",TIGERMARKET #203,"3200 Clarksville Highway, Nashville","['gas_station', 'point_of_interest', 'establis...",Clarksville Pike & S Hamilton Rd SB,United States,"['transit_station', 'point_of_interest', 'esta...",Shoe Show,"Bordeaux S, 3209 Clarksville Pike C, Nashville","['shoe_store', 'point_of_interest', 'store', '...",Excel Builders of Tennessee,"2107 Buena Vista Pike, Nashville","['general_contractor', 'real_estate_agency', '...",Campus Rx,"2109 Buena Vista Pike, Nashville","['health', 'point_of_interest', 'establishment']",3200,True,False,False,False,False,1,TIGERMARKET #203,"3200 Clarksville Highway, Nashville","['gas_station', 'point_of_interest', 'establis...",80,"3200 Clarksville Pike, Nashville",1
2,2,"(36.16502, -86.78968)",36.165020,-86.789680,"130 LifeWay Plaza, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",Frankie Pierce Park,"130 LifeWay Plaza, Nashville","['park', 'point_of_interest', 'establishment']",Frankie Pierce Park,"Frankie Pierce Park, 130 LifeWay Plaza, Nashville","['park', 'point_of_interest', 'establishment']",Rukus Cycling Studios,"1008 Nelson Merry Street, Nashville","['point_of_interest', 'establishment']",Mentoring in Motion,"460 10th Circle North, Nashville","['point_of_interest', 'establishment']","Choral Arts Link, Inc.","460 10th Circle North, Nashville","['point_of_interest', 'establishment']",130,True,False,False,False,False,1,Frankie Pierce Park,"130 LifeWay Plaza, Nashville","['park', 'point_of_interest', 'establishment']",100,"130 LifeWay Plaza, Nashville",4
3,3,"(36.2328375, -86.75830260000001)",36.232838,-86.758303,"3131 Dickerson Rd, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",Wendy's,"3131 Dickerson Road, Nashville","['restaurant', 'food', 'point_of_interest', 'e...",Maplewood Trace & Dickerson PK WB,United States,"['transit_station', 'point_of_interest', 'esta...",Church's Chicken,"3127 Dickerson Pike, Nashville","['restaurant', 'food', 'point_of_interest', 'e...",Econo Lodge North,"110 Maplewood Trace, Nashville","['lodging', 'point_of_interest', 'establishment']",TIGERMARKET #201,"3144 Dickerson Pike, Nashville","['gas_station', 'point_of_interest', 'establis...",3131,True,False,False,False,False,1,Wendy's,"3131 Dickerson Road, Nashville","['restaurant', 'food', 'point_of_interest', 'e...",96,"3131 Dickerson Rd, Nashville",4
4,4,"(36.1105402, -86.7564862)",36.110540,-86.756486,"651 Thompson Ln, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",Tennessee Glass Co,"651 Thompson Lane, Nashville","['furniture_store', 'home_goods_store', 'gener...",Nadeau - Furniture With a Soul,"647 Thompson Lane, Nashville

In [4]:
covid_data_70 = covid_data[covid_data['ratio'] >= 70]
covid_data_70.shape

(1637, 34)

In [5]:
covid_top_10 = covid_data_70.sort_values('Request #',ascending=False).head(10)

In [6]:
covid_top_10[['final_name','final_type','Request #','lat','lng']]

,final_name,final_type,Request #,lat,lng
243,Kid Rock's Big Ass Honky Tonk Rock N' Roll Ste...,"['restaurant', 'food', 'point_of_interest', 'e...",47,36.161380,-86.775610
56,Scoreboard Bar & Grill,"['bar', 'point_of_interest', 'establishment']",43,36.218237,-86.697950
684,Jonathan's Grille,"['bar', 'restaurant', 'food', 'point_of_intere...",42,36.170861,-86.782343
311,Chopper's Grill,"['bar', 'restaurant', 'food', 'point_of_intere...",32,36.264200,-86.651201
153,Solo Style LLC,"['clothing_store', 'point_of_interest', 'store...",29,36.068730,-86.683810
613,Tootsies Orchid Lounge,"['tourist_attraction', 'bar', 'restaurant', 'f...",27,36.160696,-86.778194
670,Winners Bar & Grill,"['bar', 'restaurant', 'food', 'point_of_intere...",26,36.149571,-86.796822
104,Super hair world,"['hair_care', 'point_of_interest', 'store', 'e...",22,36.258654,-86.713114
108,Metal Moulding Corporation,"['point_of_interest', 'establishment']",21,36.285234,-86.688050
157,Sinkers Beverages,"['liquor_store', 'point_of_interest', 'store',...",21,36.209568,-86.733704


In [19]:
fig = px.scatter_mapbox(covid_data, lat="lat", lon="lng", 
                        size= "Request #",
                        color="Request #",
                        opacity=0.5,
                        color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=15, zoom=10,
                        mapbox_style="carto-positron")
fig.show()

In [20]:
fig = px.scatter_mapbox(covid_top_10, lat="lat", lon="lng", 
                         size= "Request #",
                        color="Request #",
                        opacity=0.5,
                        color_continuous_scale=px.colors.cyclical.IceFire, 
                        size_max=15, zoom=10,
                        mapbox_style="carto-positron")
fig.show()